In [ ]:
!kg download -u rdugue89 -p 6fqCZ3qpN4rAKGRNRRH3 -c humpback-whale-identification

In [ ]:
%config IPCompleter.greedy=True

In [ ]:
import os
import pandas as pd
import numpy as np
from fastai.vision import *
from fastai.basics import *
import random
import shutil 
from fastai.metrics import accuracy
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [ ]:
torch.__version__

In [ ]:
path = './data'
tr_path = f'{path}/train'
test_path = f'{path}/test'

In [ ]:
df = pd.read_csv(f'{path}/train.csv')
df.head()

In [ ]:
rows = len(df.index)
tr_path = f'{path}/train'

for r in range(rows):
    directory = f'{tr_path}/{df.iloc[r][1]}'
    old_file = f'{tr_path}/{df.iloc[r][0]}'
    new_file = f'{directory}/{df.iloc[r][0]}'
    if not os.path.exists(directory):
        os.makedirs(directory)
    if os.path.exists(old_file):
        shutil.move(old_file, new_file)

In [ ]:
import string

tfms = get_transforms()

def get_img(r):
    return open_image(f'{path}/{df.iloc[r][0]}')

def get_sample_size(l):
    directory = f'{tr_path}/{l}'
    return len([f for f in os.listdir(directory) if os.path.isfile(f)])

def get_category_list(l):
    directory = f'{tr_path}/{l}'
    imgs = [f'{directory}/{df.iloc[r][0]}' for r in range(rows) if df.iloc[r][1] == l]
    return [open_image(imgs[i]) for i in range(len(imgs))]

def do_random_tfms(l):
    directory = f'{tr_path}/{l}'
    imgs = get_category_list(l)
    while len(imgs) < 5:
        imgs.append(imgs[random.randint(0,len(imgs)-1)].apply_tfms(tfms[random.randint(0,len(tfms)-1)]))
        name = ''.join(random.choices(string.ascii_uppercase + string.digits, k=9))
        save_pth = f'{directory}/{name}.jpg'
        imgs[-1].save(save_pth)
    
for label in df.iloc[:, 1].tolist():
    if label != 'new_whale':
        if get_sample_size(label) < 5:
            do_random_tfms(label)

In [ ]:
test_imgs = [f for f in os.listdir(test_path) if os.path.isfile(os.path.join(test_path, f))]

In [ ]:
data = (ImageItemList.from_folder(path)
        .random_split_by_pct()
        .label_from_folder()
        .transform(None, size=256)
        .add_test_folder(test_path)
        .databunch(bs=64)
        .normalize(imagenet_stats)) 

In [ ]:
data.show_batch(rows=4)

In [ ]:
learn = create_cnn(data, models.resnet50, metrics=accuracy)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, 1e-02)

In [ ]:
learn.save('whales')

In [ ]:
learn.load('whales')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-5,3e-4), pct_start=0.05)

In [ ]:
def top_5_preds(preds): return np.argsort(preds.numpy())[:, ::-1][:, :5]

def top_5_pred_labels(preds, classes):
    top_5 = top_5_preds(preds)
    labels = []
    for i in range(top_5.shape[0]):
        labels.append(' '.join([classes[idx] for idx in top_5[i]]))
    return labels

def create_submission(preds, data, name, classes=None):
    if not classes: classes = data.classes
    sub = pd.DataFrame({'Image': [path.name for path in data.test_ds.x.items]})
    sub['Id'] = top_5_pred_labels(preds, classes)
    sub.to_csv(f'subs/{name}.csv.gz', index=False, compression='gzip')

In [ ]:
preds, _ = learn.get_preds(DatasetType.Test)

In [ ]:
name = 'preds-1'
create_submission(preds, learn.data, name)

In [ ]:
pd.read_csv(f'submissions /{name}.csv.gz').head()